# Предсказание оттока пользователей телеком компании

В данном исследовании моделируется отток пользователей телеком компании с помощью различных моделей машинного обучения с целью определения лучшей модели. В качестве метрики качества используется ROC AUC.

Данные для исследования взяты с платформы [Kaggle](https://www.kaggle.com/c/advanced-dls-spring-2021/).


**Структура данных:**

Информация о пользователях телеком компании находится в папке `data`:

`data/train.csv` - обучающая выборка:

* `ClientPeriod` – количество полных месяцев с момента регистрации пользователя


* `MonthlySpending` – средние расходы пользователя в месяц


* `TotalSpent` – общее количество денег, потраченых клиентом


* `Sex` – пол пользователя


* `IsSeniorCitizen` – является ли клиент пенсионером


* `HasPartner` – наличие у клиента супруга(-и)


* `HasChild` – наличие у клиента детей


* `HasPhoneService` – подключение телефонного сервиса


* `HasMultiplePhoneNumbers` – подключения нескольких телефонных номеров


* `HasInternetService` – подключение интернет сервиса


* `HasOnlineSecurityService` – подключение сервиса по онлайн безопасности


* `HasOnlineBackup` – наличие облачной резервировной копии данных


* `HasDeviceProtection` – наличие защиты устройства


* `HasTechSupportAccess` – подключение услуги технической поддержки


* `HasOnlineTV` – подключение онлайн телевидения


* `HasMovieSubscription` – наличие подписки на онлайн-кинотеатр


* `HasContractPhone` – тип контракта для телефонного сервиса


* `IsBillingPaperless` – факт безналичной оплаты


* `PaymentMethod` – метод оплаты


* `Churn` – факт ухода пользователя


`data/test.csv` - тестовая выборка, её схема аналогична обучающей выборке.


**План:**

# Setup

In [ ]:
import re

import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import sklearn
from catboost import CatBoostClassifier
from ipywidgets import fixed, interact
from lightgbm import LGBMClassifier
from matplotlib_inline.backend_inline import set_matplotlib_formats
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedShuffleSplit,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.tree import DecisionTreeClassifier
from tqdm.notebook import tqdm

In [ ]:
np.random.seed(42)
pd.set_option("display.float_format", "{:,.3f}".format)
set_matplotlib_formats("svg")
pio.templates.default = "plotly_white"
pio.templates["plotly_white"]["layout"]["font"] = {"size": 14, "color": "black"}

# Исследование данных

In [ ]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

In [ ]:
interact(lambda data: display(globals()[data + "_data"]), data=["train", "test"]);

Приведем названия столбцов к snake_case стилю.

In [ ]:
def reformat_column_names(column_name):
    return "_".join(re.findall("[A-Z][^A-Z]*", column_name)).lower().replace("t_v", "tv")

In [ ]:
train_data.columns = map(reformat_column_names, train_data.columns)
test_data.columns = map(reformat_column_names, test_data.columns)

Посмотрим на общую информацию о данных.

In [ ]:
interact(lambda df: globals()[df].info(memory_usage="deep"), df=["train_data", "test_data"]);

В данных нет явных пропусков, однако столбец `total_spent` почему-то имеет тип данных `object`, хотя в нем записаны числовые значения. Проверим, нет ли в нем неявные пропущенные значения.

In [ ]:
train_data.query("total_spent in ['', ' ', 'None', 'nan']")

Действительно, в данных есть неявные пропуски, обозначенные как `' '`. Заменим их и преобразуем столбец к типу `float`.

In [ ]:
train_data.replace(" ", np.nan, inplace=True)
test_data.replace(" ", np.nan, inplace=True)

In [ ]:
train_data["total_spent"] = pd.to_numeric(train_data["total_spent"], downcast="float")
test_data["total_spent"] = pd.to_numeric(test_data["total_spent"], downcast="float")

Данные конвертировались в тип `float` без ошибок, значит, других нечисленных значений нет. Посмотрим на количество пропусков.

In [ ]:
interact(lambda data: globals()[data + "_data"].isna().sum(), data=["train", "test"]);

Всего 9 и 2 пропущенных значения в обучающей и тестовой выборке соотвественно. При построении моделей заполним их средними значениями среди соотвествующих столбцов.

Разделим данные на категориальные и количественные.

In [ ]:
categorical_columns = [column for column in train_data.columns if train_data[column].nunique() < 20]
numeric_columns = train_data.columns.difference(categorical_columns).tolist()

## Категориальные данные

In [ ]:
len(categorical_columns)

In [ ]:
def get_categorical_data_distribution_pieplot(data, rows, cols, rotations=None, height=None):
    fig = make_subplots(
        rows,
        cols,
        specs=[[{"type": "domain"}] * cols] * rows,
        vertical_spacing=0.08,
        horizontal_spacing=0.1,
        subplot_titles=data.columns,
    )

    rotations = rotations if rotations else [0] * data.shape[1]

    for i, (column, rotation) in enumerate(zip(data.columns, rotations)):
        value_counts = data[column].value_counts().reset_index()
        value_counts.columns = [column, "percent"]

        feature_fig = go.Pie(
            labels=value_counts[column],
            values=value_counts["percent"],
            rotation=rotation,
            marker_colors=pio.templates["plotly_white"]["layout"]["colorway"],
            name=column,
        )
        feature_fig.update(textinfo="label+percent", insidetextfont_color="white", insidetextorientation="horizontal")

        fig.add_trace(feature_fig, row=i // cols + 1, col=i % cols + 1)

    fig.update_layout(showlegend=False, font_size=10, margin=dict(l=0, r=0, b=0, t=50), height=height)

    for annotation in fig.layout.annotations:
        annotation.update(y=annotation.y + 0.01, font_size=10, text="<b>" + annotation.text + "</b>")

    return fig

In [ ]:
def categorical_distribution_by_dataset(data):
    rotations = [0] * 4 + [-30] + [0] * 8 + [45] + [0] + [-110] + [0]
    if data == "train":
        return get_categorical_data_distribution_pieplot(train_data[categorical_columns], 5, 4, rotations, height=1000)
    if data == "test":
        return get_categorical_data_distribution_pieplot(
            train_data[categorical_columns[:-1]], 4, 4, rotations, height=880
        )

In [ ]:
interact(categorical_distribution_by_dataset, data=["train", "test"]);

Распределения обучающей и тестовой выборки одинаковые, а в целевой переменной присутствует значительный дисбаланс классов.

## Числовые данные

In [ ]:
len(numeric_columns)

In [ ]:
def get_numeric_data_distribution_histogram():
    fig = make_subplots(2, 3, subplot_titles=numeric_columns, row_titles=["train_data", "test_data"])

    for i, (column, color) in enumerate(
        zip(numeric_columns, pio.templates["plotly_white"]["layout"]["colorway"]), start=1
    ):
        train_feature_fig = go.Histogram(x=train_data[column], marker_color=color)
        test_feature_fig = go.Histogram(x=test_data[column], marker_color=color)
        fig.add_trace(train_feature_fig, row=1, col=i)
        fig.add_trace(test_feature_fig, row=2, col=i)

    fig.update_layout(showlegend=False, margin=dict(r=0, b=0, t=50), height=300)

    for annotation in fig.layout.annotations:
        if annotation.text in numeric_columns:
            annotation.update(y=annotation.y + 0.01)
        if annotation.text in ["train_data", "test_data"]:
            annotation.update(x=-0.09, textangle=-90)

    return fig

In [ ]:
get_numeric_data_distribution_histogram()

Количественные распределения выборок тоже практически одинаковые.

# Предсказание оттока

Мы попробуем много различных моделей и посмотрим, какой результат они дадут.

In [ ]:
X = train_data.drop("churn", axis=1)
y = train_data["churn"]

Разобьем `train_data` на обучающую и валидационную выборки. Так как в данных присутствует дисбаланс классов, выборки сделаем стратифицированными.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, stratify=y, shuffle=True)

In [ ]:
# удаление столбца churn
categorical_columns.pop();

Прежде, чем передавать данные в модель, сначала избавимся от пропусков в них, нормализируем числовые значения и закодируем категориальные.

In [ ]:
def create_model(model: type, encoder: sklearn.preprocessing._encoders = None, *args, **kwargs):

    encoder = encoder if encoder else OneHotEncoder(handle_unknown="ignore")

    numeric_preprocessor = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="median")),
            ("standard_scaler", StandardScaler()),
        ]
    )

    categorical_preprocessor = Pipeline(
        [
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("one_hot_encoder", encoder),
        ]
    )

    preprocessor = ColumnTransformer(
        [
            ("numeric_preprocessor", numeric_preprocessor, numeric_columns),
            ("categorical_preprocessor", categorical_preprocessor, categorical_columns),
        ],
        n_jobs=-1,
    )

    model = Pipeline([("preprocessor", preprocessor), ("classifier", model(*args, **kwargs))])

    return model

## Метод k-ближайших соседей

Сначала будем тестировать одну из самых простых моделей - метод k-ближайших соседей.

In [ ]:
knn_model = create_model(KNeighborsClassifier, n_jobs=-1)
knn_model

Посмотрим на базовое качество модели.

In [ ]:
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

Невысокая точность и низкие precision и recall – последние два нам интересны даже больше, потому что у нас дисбаланс классов. Посмотрим на кросс-валидацию.

In [ ]:
cv = StratifiedShuffleSplit(n_splits=10, train_size=0.8)

cv_results_base = pd.DataFrame(cross_validate(knn_model, X, y, cv=cv, n_jobs=-1, scoring=("accuracy", "f1", "roc_auc")))
cv_results_base

In [ ]:
cv_results_base[["test_f1", "test_roc_auc"]].mean()

Качество модели стабильно невысокое, F1 на валидационной выборке не превышает 0.6.

In [ ]:
pd.DataFrame(confusion_matrix(y_val, y_pred), index=["y_true_0", "y_true_1"], columns=["y_pred_0", "y_pred_1"])

Попытаемся улучшить качество модели, изменив гиперпараметры. Для этого будем использовать RandomSearch. В отличие от GridSearch он позволяет сэкономить время и работает над большим полем гиперпараметров.

### Случайный поиск

Для некоторых параметров нужны целочисленные значения, так что создадим функцию для преобразования распределений модуля `scipy.stats` в целочисленные.

In [ ]:
def create_int_distribution(distribution):
    class int_distribution:
        def __init__(self, *args, **kwargs):
            self._distribution = distribution(*args, **kwargs)

        def rvs(self, *args, **kwargs):
            return self._distribution.rvs(*args, **kwargs).astype(int)

    return int_distribution

In [ ]:
uniform_int = create_int_distribution(stats.uniform)

In [ ]:
knn_param_distributions = dict(
    n_neighbors=uniform_int(5, 64),
    weights=["uniform", "distance"],
    algorithm=["auto", "ball_tree", "kd_tree", "brute"],
    leaf_size=uniform_int(5, 64),
    p=[1, 2],
)

knn_param_names = list(knn_param_distributions.keys())

knn_param_distributions = {"classifier__" + key: value for key, value in knn_param_distributions.items()}

In [ ]:
knn_model_random_search = RandomizedSearchCV(
    knn_model, knn_param_distributions, n_iter=100, scoring=("f1", "roc_auc"), refit="f1", n_jobs=-1
)

Протестируем на кросс-валидации.

In [ ]:
%%time
cv = StratifiedShuffleSplit(n_splits=5, train_size=0.8)
cv_results_random_search = pd.DataFrame(
    cross_validate(knn_model_random_search, X, y, cv=cv, scoring=("f1", "roc_auc"), n_jobs=-1)
)
cv_results_random_search

In [ ]:
cv_results = pd.DataFrame(
    [
        cv_results_base[["test_f1", "test_roc_auc"]].mean(),
        cv_results_random_search[["test_f1", "test_roc_auc"]].mean(),
    ],
    index=["base", "random_search"],
).T

cv_results

Метрики F1 и ROC AUC увеличились на ~0.05. Попробуем искать больше различных вариантов.

In [ ]:
knn_model_random_search = RandomizedSearchCV(
    knn_model, knn_param_distributions, n_iter=100, scoring=("f1", "roc_auc"), refit="f1", n_jobs=-1
)

Делать вложенную кросс-валидацию для RandomSearch с 500 итерациями слишком времязатратно, так что обойдемся без нее.

In [ ]:
%%time
knn_model_random_search.fit(X_train, y_train)
y_pred = knn_model_random_search.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

In [ ]:
y_pred_proba = knn_model_random_search.predict_proba(X_val)
roc_auc_score(y_val, y_pred_proba[:, 1])

Смогли еще немного увеличить качество модели. В итоге лучшее качество для модели k-ближайших соседей:

**F1:** 0.618
<br>
**ROC AUC:** 0.847

Теперь посмотрим, как выбор различных гиперпараметров влияет на качество модели.

In [ ]:
knn_random_search_cv_results = pd.DataFrame(knn_model_random_search.cv_results_)
knn_random_search_cv_results.columns = knn_random_search_cv_results.columns.str.replace("param_classifier__", "")
knn_random_search_cv_results = knn_random_search_cv_results[knn_param_names + ["mean_test_f1", "mean_test_roc_auc"]]
knn_random_search_cv_results = knn_random_search_cv_results.apply(pd.to_numeric, errors="ignore")
knn_random_search_cv_results["p"] = knn_random_search_cv_results["p"].astype(object)

In [ ]:
knn_random_search_cv_results.head()

In [ ]:
def create_features_parcoord(cv_results, score_name, only_top=False):
    dimensions = []
    label_encoder = LabelEncoder()

    for (column_name, column) in cv_results.iteritems():
        if only_top and column_name == score_name:
            constraintrange = [column.max() * 0.98, column.max()]
        else:
            constraintrange = None

        if column.dtype == object:
            dimension = dict(
                range=[-1, column.nunique()],
                label=column_name,
                values=label_encoder.fit_transform(column),
                tickvals=np.arange(0, column.nunique()),
                ticktext=label_encoder.classes_,
            )
        else:
            dimension = dict(
                range=[column.min(), column.max()],
                constraintrange=constraintrange,
                label=column_name,
                values=column,
            )
        dimensions.append(dimension)

    fig = go.Figure(
        go.Parcoords(
            line=dict(
                color=cv_results[score_name],
                colorscale="Plasma",
                showscale=True,
                cmin=cv_results[score_name].min() * 1.1,
                cmax=cv_results[score_name].max(),
                colorbar=dict(title=dict(text=score_name, font_size=16, side="right"), tickfont_size=12),
            ),
            dimensions=dimensions,
        )
    )

    return fig

In [ ]:
def create_features_parcoord_by_metric(random_search_cv_results, metric, only_top=False):
    if metric == "mean_test_f1":
        return create_features_parcoord(
            random_search_cv_results.drop("mean_test_roc_auc", axis=1), "mean_test_f1", only_top
        )
    if metric == "mean_test_roc_auc":
        return create_features_parcoord(
            random_search_cv_results.drop("mean_test_f1", axis=1), "mean_test_roc_auc", only_top
        )

In [ ]:
interact(
    create_features_parcoord_by_metric,
    random_search_cv_results=fixed(knn_random_search_cv_results),
    metric=["mean_test_f1", "mean_test_roc_auc"],
    only_top=[False, True],
);

In [ ]:
fig = create_features_parcoord(
    cv_results=knn_random_search_cv_results.drop("mean_test_roc_auc", axis=1), score_name="mean_test_f1"
)
fig.write_html("fig.html")

У топ 2% лучших моделей используется равномерное усреднение соседей, а не взвешенное. Остальные параметры не сильно влияют на качество модели.

## Логистическая регрессия

Посмотрим, как с задачей справится линейная модель.

In [ ]:
lr_model = create_model(LogisticRegressionCV, scoring="f1", max_iter=1000, n_jobs=-1)
lr_model

In [ ]:
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

Логистическая регрессия кажется немного лучше метода k-ближайших соседей. Посмотрим на кросс-валидацию.

In [ ]:
cv = StratifiedShuffleSplit(n_splits=10, train_size=0.8)

cv_results_base = pd.DataFrame(cross_validate(lr_model, X, y, cv=cv, n_jobs=-1, scoring=("f1", "roc_auc")))
cv_results_base

In [ ]:
cv_results_base[["test_f1", "test_roc_auc"]].mean()

Значения метрик у базовой логистической регрессии чуть хуже, чем значения у оптимизированной модели k-ближайших соседей. Попробуем отыскать оптимальные параметры модели. У логистической регрессии параметров не так много, так что можно использовать GridSearch, чтобы перебрать все комбинации.

### Поиск по сетке

In [ ]:
lr_param_grid = dict(
    penalty=["l1", "l2", "elasticnet"],
    solver=["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
    class_weight=[None, "balanced"],
)

lr_param_names = list(lr_param_grid.keys())

lr_param_grid = {"classifier__" + key: value for key, value in lr_param_grid.items()}

In [ ]:
lr_model_grid_search = GridSearchCV(lr_model, lr_param_grid, scoring=("f1", "roc_auc"), refit="f1", n_jobs=-1)

Из-за наличия несовместимых параметров, GridSearch будет выкидывать ошибку обучения соотвествующей модели. Я не смог найти, как отключить эти предупреждения, и судя по всему это баг, связанный с мультипроцессингом и параметром `n_jobs=-1`.

In [ ]:
%%time
# from sklearn.utils._testing import ignore_warnings
# from sklearn.exceptions import FitFailedWarning, ConvergenceWarning
# with ignore_warnings(category=[ConvergenceWarning, FitFailedWarning]):
cv = StratifiedShuffleSplit(n_splits=5, train_size=0.8)

cv_results_grid_search = pd.DataFrame(
    cross_validate(lr_model_grid_search, X, y, cv=cv, scoring=("f1", "roc_auc"), n_jobs=-1)
)

In [ ]:
%mprun cv_results_grid_search

In [ ]:
cv_results_grid_search[["test_f1", "test_roc_auc"]].mean()

Оптимизированная логистическая регрессия показывает примерно те же результаты, что и метод k-ближайших соседей.
<br>

Лучшее качество для логистической регрессии:

**F1:** 0.622
<br>
**ROC AUC:** 0.847

## Решающее дерево

In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
dt_model = create_model(DecisionTreeClassifier, encoder=ordinal_encoder)
dt_model

In [ ]:
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

In [ ]:
y_pred_proba = dt_model.predict_proba(X_val)
roc_auc_score(y_val, y_pred_proba[:, 1])

Модель решающего дерева с параметрами по умолчанию хуже предыдущих двух. Будем оптимизировать. В этот раз будем использовать библиотеку **Optuna**, реализующую различные методы оптимизации, такие как баесовская оптимизация, алгоритм имитации отжига и другие. Прежде, чем оптимизировать, с помощью RandomSearch посмотрим на то, как разные параметры влияют на качество модели.

### Случайный поиск

In [ ]:
dt_param_distributions = dict(
    criterion=["gini", "entropy", "log_loss"],
    max_depth=uniform_int(8, 128),
    min_samples_split=uniform_int(2, 128),
    min_samples_leaf=uniform_int(1, 128),
    max_features=[None, "sqrt", "log2"],
    class_weight=[None, "balanced"],
)

dt_param_names = list(dt_param_distributions.keys())

dt_param_distributions = {"classifier__" + key: value for key, value in dt_param_distributions.items()}

In [ ]:
dt_model_random_search = RandomizedSearchCV(
    dt_model,
    dt_param_distributions,
    n_iter=5000,
    scoring=("f1", "roc_auc"),
    refit="f1",
    return_train_score=True,
    n_jobs=-1,
)

In [ ]:
%%time
dt_model_random_search.fit(X_train, y_train)
y_pred = dt_model_random_search.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

In [ ]:
y_pred_proba = dt_model_random_search.predict_proba(X_val)
roc_auc_score(y_val, y_pred_proba[:, 1])

У модели значительно увеличился recall, но практически не вырос precision.

In [ ]:
dt_random_search_cv_results = pd.DataFrame(dt_model_random_search.cv_results_)
dt_random_search_cv_results.columns = dt_random_search_cv_results.columns.str.replace("param_classifier__", "")
dt_random_search_cv_results = dt_random_search_cv_results[dt_param_names + ["mean_test_f1", "mean_test_roc_auc"]]
dt_random_search_cv_results = dt_random_search_cv_results.apply(pd.to_numeric, errors="ignore")
dt_random_search_cv_results.fillna("None", inplace=True)

In [ ]:
dt_random_search_cv_results.head()

In [ ]:
interact(
    create_features_parcoord_by_metric,
    random_search_cv_results=fixed(dt_random_search_cv_results),
    metric=["mean_test_f1", "mean_test_roc_auc"],
);

Все лучшие модели имеют параметр `class_weight` равный `balanced`. В остальном выбор параметров не сильно влияет на качество модели. Попробуем улучшить результат с помощью баесовской оптимизации.

### Баесовская оптимизация

In [ ]:
from ray.tune.suggest.optuna import OptunaSearch

In [ ]:
def objective(trial):

    params = dict(
        class_weight=trial.suggest_categorical("class_weight", [None, "balanced"]),
        max_features=trial.suggest_categorical("max_features", [None, "log2", "sqrt"]),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 20, 100),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 120),
        max_depth=trial.suggest_int("max_depth", 8, 120),
        criterion=trial.suggest_categorical("criterion", ["entropy", "gini", "log_loss"]),
    )

    dt_model.set_params(**{"classifier__" + key: value for key, value in params.items()})

    cv = StratifiedShuffleSplit(n_splits=10, train_size=0.8)
    try:
        cv_results = pd.DataFrame(
            cross_validate(dt_model, X_train, y_train, cv=cv, n_jobs=-1, scoring=("f1", "roc_auc"))
        )
        f1_score = cv_results["test_f1"].mean()
        return f1_score
    except Exception:
        return 0

In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
dt_model = create_model(DecisionTreeClassifier, encoder=ordinal_encoder, class_weight="balanced")
study = optuna.create_study(study_name="decision_tree_optimization", direction="maximize")
study.optimize(objective, n_trials=2000, show_progress_bar=True)

In [ ]:
study.best_value

In [ ]:
trials = study.trials_dataframe()
trials.columns = trials.columns.str.replace("params_", "").str.replace("value", "mean_test_f1")
trials = trials[dt_param_names[:-1] + ["mean_test_f1"]].fillna("None")

In [ ]:
trials.head()

In [ ]:
create_features_parcoord(trials, "mean_test_f1")

In [ ]:
dt_model.set_params(**{"classifier__" + key: value for key, value in study.best_params.items()})

In [ ]:
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

In [ ]:
y_pred_proba = dt_model.predict_proba(X_val)
roc_auc_score(y_val, y_pred_proba[:, 1])

## Случайный лес

## Градиентный бустинг
